In [28]:
import os
import sys
BASE_DIR = r"C:\Users\nino.zhou\Desktop\my_project\cp_newbank_mobile_behavior"
sys.path.insert(0, BASE_DIR)

import pandas as pd
import numpy as np
from config import setting

In [29]:
df_train = pd.read_csv(setting.BASE_DIR + "\data_model\data\sensor_train.csv")
df_train

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0
...,...,...,...,...,...,...,...,...,...
425354,7291,4561,-0.1,-0.5,-1.7,-0.2,3.4,7.4,18
425355,7291,4647,0.7,-1.9,1.1,0.5,1.7,10.7,18
425356,7291,4735,1.3,-1.4,-0.5,1.4,2.1,8.0,18
425357,7291,4830,1.2,-1.6,-0.1,1.6,1.8,9.2,18


# 1. 数据观察

In [30]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425359 entries, 0 to 425358
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   fragment_id  425359 non-null  int64  
 1   time_point   425359 non-null  int64  
 2   acc_x        425359 non-null  float64
 3   acc_y        425359 non-null  float64
 4   acc_z        425359 non-null  float64
 5   acc_xg       425359 non-null  float64
 6   acc_yg       425359 non-null  float64
 7   acc_zg       425359 non-null  float64
 8   behavior_id  425359 non-null  int64  
dtypes: float64(6), int64(3)
memory usage: 29.2 MB


In [31]:
df_train.describe()

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id
count,425359.000000,425359.000000,425359.000000,425359.000000,425359.000000,425359.000000,425359.000000,425359.000000,425359.000000
mean,3647.035972,2500.952026,0.009635,-0.063114,0.024548,0.965656,2.730190,7.825065,8.535035
std,2103.253140,1442.307956,0.619285,0.537614,0.717009,3.295531,2.978134,2.653261,5.395264
min,0.000000,0.000000,-14.700000,-14.700000,-12.600000,-19.800000,-16.900000,-8.900000,0.000000
25%,1822.000000,1252.000000,-0.100000,-0.100000,-0.200000,-0.700000,0.400000,7.300000,4.000000
50%,3646.000000,2501.000000,0.000000,0.000000,0.000000,0.200000,2.700000,8.700000,8.000000
75%,5466.000000,3749.000000,0.100000,0.100000,0.200000,2.000000,4.900000,9.400000,13.000000
max,7291.000000,4999.000000,34.500000,13.400000,28.200000,30.200000,14.400000,38.100000,18.000000


In [32]:
# 缺失率
def df_missing_rate_count(df):
    def func(num):
        return "%.2f%%" % num.values[0]
    df_missing_rate = pd.DataFrame(df.isnull().sum(axis=0)/len(df), columns=["缺失率"]).sort_values("缺失率", ascending=False)
    df_missing_rate["缺失率"] = df_missing_rate[["缺失率"]].apply(func, axis=1)
    return df_missing_rate
df_missing_rate_count(df_train)

,缺失率
fragment_id,0.00%
time_point,0.00%
acc_x,0.00%
acc_y,0.00%
acc_z,0.00%
acc_xg,0.00%
acc_yg,0.00%
acc_zg,0.00%
behavior_id,0.00%


### 目标列behavior字段

In [35]:
df_temp = df_train.groupby("behavior_id")[["fragment_id"]].count()
df_temp.columns=["sample_count"]
df_temp

,sample_count
behavior_id,
0,13865
1,39567
2,23745
3,19392
4,25012
5,19987
6,43646
7,13864
8,19250


 - 各类别样本大致均衡，但14类别样本偏少，但也在可接受范围内

In [40]:
df_temp = df_train.groupby("fragment_id").apply(lambda x: x["behavior_id"].iloc[0]).reset_index()
df_temp.columns = ["fragment_id", "behavior_id"]
df_temp = df_temp.groupby("behavior_id")[["fragment_id"]].count()
df_temp.columns=["sample_count"]
df_temp

,sample_count
behavior_id,
0,241
1,676
2,409
3,332
4,435
5,343
6,742
7,236
8,328


 - 按照fragment_id合并后，统计不同的目标类别各自有多少个fragment_id，我们发现分布也是差不多的，还是14类别的fragment_id偏少

In [ ]:
df_temp = df_train.groupby("behavior_id")[["fragment_id"]].count()
df_temp.columns=["sample_count"]
df_temp

### fragnebt_id字段

In [23]:
df_train["fragment_id"].value_counts().reset_index().sort_values("fragment_id")

,index,fragment_id
7291,3230,50
7283,5359,50
7284,5356,50
7285,5358,50
7290,6752,50
...,...,...
7,5470,61
5,1644,61
6,2796,61
4,1645,61


 - 一共7292个不同的fragment_id，每个id有50~61条数据，frament_id对应的是一个行为片段

In [7]:
df_groupby = df_train.groupby("fragment_id").apply(lambda df_groupby: df_groupby.drop_duplicates("behavior_id").shape[0]).reset_index()
df_groupby.columns = ["fragment_id", "distinct_category"]
df_groupby[df_groupby["distinct_category"] > 1]

,fragment_id,distinct_category


 - 每个fragment_id内对应的记录，都是用户做某一个行为留下来的一系列动作，即一个用户做了某个行为，用时5s，这5s产生了一系列的加速器的数据，但都是同一个行为。所以一个fragment_id对一个只有一个行为。

In [8]:
df_groupby = df_train.groupby("fragment_id").apply(lambda df_groupby: df_groupby.drop_duplicates("time_point").shape[0]).reset_index()
df_groupby.columns = ["fragment_id", "distinct_time_point"]
df_groupby

,fragment_id,distinct_time_point
0,0,57
1,1,56
2,2,57
3,3,55
4,4,55
...,...,...
7287,7287,59
7288,7288,58
7289,7289,58
7290,7290,58


 - 每个fragment_id内对应的记录，都包含60个左右的time_point，即用户的一系列行为的时间点，简单理解就是5s内，每隔几百毫秒就采集一个数据

In [11]:
df_groupby["distinct_time_point"].value_counts()

60    2600
58    1928
57    1346
59     751
56     224
54     179
55     151
53      69
52      17
51      10
50       9
61       8
Name: distinct_time_point, dtype: int64

 - 大多数fragment的时间点数据量都在60个左右

### time_point字段

In [17]:
df_train["time_point"].value_counts()

1918    141
646     129
812     127
3162    125
728     125
       ... 
2784     55
278      55
3122     54
611      53
3038     52
Name: time_point, Length: 5000, dtype: int64

In [18]:
df_train["time_point"].describe()

count    425359.000000
mean       2500.952026
std        1442.307956
min           0.000000
25%        1252.000000
50%        2501.000000
75%        3749.000000
max        4999.000000
Name: time_point, dtype: float64

 - time_point一共5000个不同的时间点，从0-5000

In [25]:
df_the_fragment = df_train[df_train["fragment_id"] == 7287]
df_the_fragment

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id
425068,7287,2,5.7,2.5,-1.3,11.1,-5.8,-1.3,18
425069,7287,88,1.1,-3.2,-0.1,7.0,-11.1,-0.2,18
425070,7287,172,1.4,-2.3,0.9,9.2,-8.3,0.1,18
425071,7287,256,-2.3,-5.3,-0.6,5.7,-10.1,-1.2,18
425072,7287,342,-5.6,-5.2,-0.2,2.9,-8.5,-0.8,18
425073,7287,431,-7.2,-3.1,-0.5,1.7,-7.2,-1.2,18
425074,7287,516,-5.3,-2.8,-0.1,3.2,-8.6,-0.8,18
425075,7287,600,2.4,-6.5,1.8,9.5,-14.9,-0.1,18
425076,7287,684,5.5,-7.3,7.3,10.8,-13.0,6.6,18
425077,7287,769,8.9,0.9,7.5,11.2,-7.1,1.8,18


In [23]:
df_the_fragment["time_point"].diff().value_counts()

84.0    20
85.0    18
87.0     5
86.0     5
88.0     4
90.0     2
89.0     2
92.0     1
96.0     1
Name: time_point, dtype: int64

 - 每个fragment的所有time_point，都是从0-5000逐步递增的，递增的幅度大约在85左右，少量会达到90以上

# 测试集数据分析

In [2]:
df_test = pd.read_csv(setting.BASE_DIR + "\data_model\data\sensor_test.csv")
df_test

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg
0,0,230,-1.3,0.4,0.5,-1.0,6.9,6.3
1,0,318,-1.2,0.8,1.0,-0.6,7.4,7.8
2,0,403,0.0,-0.1,-0.7,0.6,6.7,6.2
3,0,490,0.9,-0.1,0.1,1.6,6.5,6.9
4,0,574,0.4,0.1,-0.4,0.9,6.6,6.6
...,...,...,...,...,...,...,...,...
466887,7999,4618,3.6,-5.7,11.5,9.4,-12.3,7.9
466888,7999,4702,5.1,-0.7,5.5,9.0,-8.5,1.7
466889,7999,4787,2.7,1.5,4.8,6.3,-6.8,2.0
466890,7999,4874,2.0,1.4,4.6,7.0,-6.9,2.4


In [4]:
df_test.describe()

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg
count,466892.000000,466892.000000,466892.000000,466892.000000,466892.000000,466892.000000,466892.000000,466892.000000
mean,4004.142943,2500.646186,0.006555,-0.065644,0.023063,0.763515,2.890043,7.841973
std,2307.677541,1442.477261,0.598688,0.528287,0.705904,3.126467,3.037868,2.580473
min,0.000000,0.000000,-16.200000,-14.700000,-15.400000,-25.400000,-20.500000,-13.100000
25%,2005.000000,1252.000000,-0.100000,-0.100000,-0.200000,-0.800000,0.500000,7.200000
50%,4007.000000,2501.000000,0.000000,-0.000000,0.000000,0.100000,2.800000,8.700000
75%,6000.000000,3749.000000,0.100000,0.100000,0.200000,1.500000,5.100000,9.400000
max,7999.000000,4999.000000,33.000000,30.000000,61.300000,34.000000,15.800000,63.600000


In [5]:
# 缺失率
def df_missing_rate_count(df):
    def func(num):
        return "%.2f%%" % num.values[0]
    df_missing_rate = pd.DataFrame(df.isnull().sum(axis=0)/len(df), columns=["缺失率"]).sort_values("缺失率", ascending=False)
    df_missing_rate["缺失率"] = df_missing_rate[["缺失率"]].apply(func, axis=1)
    return df_missing_rate
df_missing_rate_count(df_test)

,缺失率
fragment_id,0.00%
time_point,0.00%
acc_x,0.00%
acc_y,0.00%
acc_z,0.00%
acc_xg,0.00%
acc_yg,0.00%
acc_zg,0.00%


In [6]:
df_test["fragment_id"].value_counts().reset_index().sort_values("fragment_id")

,index,fragment_id
7999,5895,50
7992,7258,50
7993,7818,50
7994,6924,50
7998,477,50
...,...,...
5,3192,61
6,6427,61
7,7856,61
4,7904,61


In [9]:
df_groupby = df_test.groupby("fragment_id").apply(lambda df_groupby: df_groupby.drop_duplicates("time_point").shape[0]).reset_index()
df_groupby.columns = ["fragment_id", "distinct_time_point"]
df_groupby

,fragment_id,distinct_time_point
0,0,55
1,1,56
2,2,56
3,3,57
4,4,57
...,...,...
7995,7995,58
7996,7996,57
7997,7997,59
7998,7998,58


In [12]:
df_groupby["distinct_time_point"].value_counts()

60    2878
58    2121
57    1589
59     770
56     243
55     141
54     141
53      75
52      15
51      10
61       9
50       8
Name: distinct_time_point, dtype: int64

In [14]:
df_the_fragment = df_test[df_test["fragment_id"] == 7996]
df_the_fragment

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg
466659,7996,46,0.4,1.9,0.9,6.3,-6.2,0.8
466660,7996,134,0.1,-1.3,1.0,5.8,-8.1,1.3
466661,7996,218,3.3,-1.0,-2.5,8.3,-10.3,-2.9
466662,7996,303,3.3,-1.5,1.0,9.5,-8.1,2.5
466663,7996,399,0.9,-1.0,-3.7,7.2,-8.4,-4.4
466664,7996,481,-2.4,-2.6,-2.2,4.9,-9.1,-2.0
466665,7996,566,-3.8,-2.5,-0.1,3.4,-8.8,-1.6
466666,7996,650,-2.3,-0.9,-0.2,6.1,-7.7,-2.3
466667,7996,735,-1.0,-3.3,-0.5,6.8,-9.5,-0.8
466668,7996,824,11.3,3.0,5.9,20.1,-3.2,4.6


In [15]:
df_the_fragment["time_point"].diff().value_counts()

85.0    13
84.0    10
86.0     6
89.0     6
88.0     5
87.0     4
92.0     2
95.0     2
90.0     2
96.0     2
98.0     1
91.0     1
97.0     1
82.0     1
Name: time_point, dtype: int64